# HW6

The dataset is from https://www.kaggle.com/datasets/willianoliveiragibin/healthcare-insurance?resource=download

In [1]:
import pymc as pm
import numpy as np
from scipy import stats
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt
from scipy.linalg import cholesky

In [6]:
# Load data from CSV
data = pd.read_csv('insurance.csv')

# Define the number of data points
n = len(data)

# Define the dimensionality of the multivariate normal distribution
p = data.shape[1]

X,y=np.zeros((n,p)),np.ones((n,1))

with pm.Model() as logistic_model:
    # Priors for the regression coefficients
    betas = pm.MvNormal('betas', mu = np.zeros((p,1)), cov = np.eye(p), shape = (p,1))
    # Linear combination of features and weights
    linear_comb = pm.math.dot(X, betas)
    # Using a logit link function
    p = pm.math.sigmoid (linear_comb)
    # Bernoulli likelihood
    y_obs = pm.Bernoulli('y_obs', p=p, observed=y)
    
with logistic_model:
    idata = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [betas]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 35 seconds.


In [8]:
np.random.seed(42)
n_samples = 1000
X = np.random.normal(size=(n_samples, 1))
true_beta = 2.5
mu = X * true_beta
w = 0.5
nu = 5

# Simulating y with a normal distribution (for simplicity)
y = np.random.normal(mu.flatten(), scale=w)

# Robust regression model with explicit lambda modeling
with pm.Model() as robust_model_with_lambda:
    # Priors
    beta = pm.Normal('beta', mu=0, sigma=10)
    nu = pm.Exponential('nu', 1/29) + 1
    lambda_i = pm.Gamma('lambda', alpha=nu/2, beta=nu/2, shape=n_samples)
    sigma_i = pm.Deterministic('sigma_i', 1 / (lambda_i))

    # Likelihood
    likelihood = pm.Normal('y', mu=X[:,0]*beta, sigma=sigma_i, observed=y)

    # Sampling
    trace = pm.sample(1000, target_accept=0.9, return_inferencedata=True)

# Identify outliers based on the posterior of lambda_i
lambda_posterior = trace.posterior['lambda'].values
lambda_mean = np.mean(lambda_posterior, axis=(0, 1))
outlier_threshold = np.quantile(lambda_mean, 0.05)
outliers = np.where(lambda_mean < outlier_threshold)[0]
print(f"Potential outliers: {outliers}")

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta, nu, lambda]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 25 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Potential outliers: [ 23  61  83 101 102 109 116 151 157 160 161 165 219 229 233 276 289 291
 325 346 355 382 413 426 453 491 495 496 539 562 591 594 597 615 638 649
 662 695 712 750 766 800 816 911 924 926 934 956 957 971]
